In [1]:
from functools import reduce
import copy
from typing import List
import numpy as np

In [2]:
class Matrix:

    def __init__(self, number_rows=0, number_columns=0, name=None):

        self.__number_rows = number_rows
        self.__number_columns = number_columns
        self.name = name
        self.matrix = [[1 + i for i in range(number_columns)] for _ in range(number_rows)]

    @property
    def number_rows(self):
        return self.__number_rows

    @number_rows.setter
    def number_rows(self, amount):
        self.__number_rows = amount

    @property
    def number_columns(self):
        return self.__number_columns

    @number_columns.setter
    def number_columns(self, amount):
        self.__number_columns = amount

    def __add__(self, other):

        if self.number_rows == other.number_rows and self.number_columns == other.number_columns:
            self.matrix = [[self.matrix[i][j] + other.matrix[i][j] for j in range(self.number_columns)]
                           for i in range(self.number_rows)]

            return self.matrix

        return "you can't sum these matrices"

    def __sub__(self, other):

        if self.number_rows == other.number_rows and self.number_columns == other.number_columns:
            self.matrix = [[self.matrix[i][j] - other.matrix[i][j] for j in range(self.number_columns)]
                           for i in range(self.number_rows)]

            return self.matrix

        return "you can't sum these matrices"

    def __mul__(self, other):

        if isinstance(other, Matrix):
            if self.number_columns == other.number_rows:

                result = [[0 for j in range(other.number_columns)] for i in range(self.number_rows)]

                for i in range(self.number_rows):
                    for j in range(other.number_columns):

                        result[i][j] = reduce(lambda x, y: x + y, [self.matrix[i][a]*other.matrix[a][j]
                                                                   for a in range(self.number_columns)], 0)
                check_result = list(np.dot(self.matrix, other.matrix))

                return result, check_result

            else:

                return "you can't mul these matrices"

        self.matrix = [[self.matrix[i][j] * other for j in range(self.number_columns)] for i in range(self.number_rows)]

        return self.matrix

    def transposition(self):

        transposition_matrix = [[0 for j in range(self.number_rows)] for i in range(self.number_columns)]

        for i in range(self.number_rows):
            for j in range(self.number_columns):
                transposition_matrix[j][i] = self.matrix[i][j]

        check_result = np.transpose(self.matrix)

        return transposition_matrix, check_result

In [3]:
class SquareMatrix(Matrix):

    _instance = None

    def __new__(cls, *args, **kwargs):

        row, colum, name = args
        if row == colum:
            cls._instance = super().__new__(cls)

            return cls._instance

        else:
            print("your matrix should be square")

    def matrix_minor(self, i: int, j: int) -> List[List[int]]:

        """
        creates the matrix to calculate the minor of the i-row and j-column
        :param i: the number of the row
        :param j: the number of the column
        :return: the matrix to calculate the minor(i, j)
        """

        if self.number_columns == 1:

            return self.matrix

        matrix_minor = copy.deepcopy(self.matrix)
        matrix_minor.pop(i)

        for row in range(self.number_columns - 1):
            matrix_minor[row].pop(j)

        return matrix_minor

In [4]:
class SecondOrderMatrix(SquareMatrix):

    def determinant(self):

        """
        calculates the second-order determinant
        :return: the second-order determinant
        """

        return self.matrix[0][0] * self.matrix[1][1] - self.matrix[1][0] * self.matrix[0][1]

In [5]:
class ThirdOrderMatrix(SquareMatrix):

    def determinant1(self):

        """
        calculates the third-order determinant
        :return: the third-order determinant
        """

        minors = [self.matrix_minor(0, j) for j in range(self.number_columns)]
        determinants = []
        result = 0

        for j in range(self.number_columns):

            matrix_second_order = SecondOrderMatrix(2, 2, None)
            matrix_second_order.matrix = minors[j]
            determinants.append(matrix_second_order.determinant())
            result += self.matrix[0][j]*matrix_second_order.determinant()*((-1)**j)

        return result

    def determinant2(self):

        """
        calculates the third-order determinant
        :return: the third-order determinant
        """

        return (self.matrix[0][0] * self.matrix[1][1] * self.matrix[2][2] +
                self.matrix[1][0] * self.matrix[2][1] * self.matrix[0][2] +
                self.matrix[0][1] * self.matrix[1][2] * self.matrix[2][0]) -\
            (self.matrix[2][0] * self.matrix[1][1] * self.matrix[0][2] +
             self.matrix[1][0] * self.matrix[0][1] * self.matrix[2][2] +
                self.matrix[2][1] * self.matrix[1][2] * self.matrix[0][0])


In [6]:
class FourthOrderMatrix(SquareMatrix):

    def determinant(self):

        """
        calculates the fourth-order determinant
        :return: the fourth-order determinant
        """

        minors = [self.matrix_minor(0, j) for j in range(self.number_columns)]
        determinants = []
        result = 0

        for j in range(self.number_columns):

            matrix_third_order = ThirdOrderMatrix(3, 3, None)
            matrix_third_order.matrix = minors[j]
            determinants.append(matrix_third_order.determinant2())
            result += self.matrix[0][j]*matrix_third_order.determinant2()*((-1)**j)

        check_result = np.linalg.det(self.matrix)

        return result, check_result

In [7]:
matrix_1 = FourthOrderMatrix(4, 4, 'matrix_1')
matrix_2 = Matrix(4, 4, 'matrix_2')

print(matrix_1.matrix)
matrix_1.matrix[0][0] = 11
matrix_1.matrix[1][1] = 4
matrix_1.matrix[0][2] = 1
matrix_1.matrix[3][3] = 10
print(matrix_1.matrix)
print(matrix_1.determinant())
print(matrix_1 - matrix_2)


[[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]]
[[11, 2, 1, 4], [1, 4, 3, 4], [1, 2, 3, 4], [1, 2, 3, 10]]
(384, 384.0)
[[10, 0, -2, 0], [0, 2, 0, 0], [0, 0, 0, 0], [0, 0, 0, 6]]
